# Analyse du data

In [85]:
# Imports PySpark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Initialisation de la session Spark
spark = SparkSession.builder \
    .appName("BTC_Silver") \
    .getOrCreate()

In [86]:
# Lecture du fichier Parquet Bronze
bronze_path = "../data/bronze/btc_minute_data.parquet"
df_bronze = spark.read.parquet(bronze_path)

# Afficher un aperçu
df_bronze.show(5)


+-------------------+--------+--------+--------+--------+-------+--------------------+------------------+----------------+---------------------+----------------------+------+
|          open_time|    open|    high|     low|   close| volume|          close_time|quote_asset_volume|number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|
+-------------------+--------+--------+--------+--------+-------+--------------------+------------------+----------------+---------------------+----------------------+------+
|2026-01-19 09:53:00|93046.35|93046.35|92997.65| 93001.8|9.10532|2026-01-19 09:53:...|    847027.2616431|            2024|              2.08878|        194272.7849138|     0|
|2026-01-19 09:54:00| 93001.8|93022.23|93001.79|93014.94|5.62556|2026-01-19 09:54:...|    523245.8306426|            1064|              3.42885|        318907.7899883|     0|
|2026-01-19 09:55:00|93014.95|93042.27|93014.95|93022.36|3.74306|2026-01-19 09:55:...|    348227.1952126|            1863|   

In [87]:
df_bronze.printSchema()

root
 |-- open_time: timestamp_ntz (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- close_time: timestamp_ntz (nullable = true)
 |-- quote_asset_volume: double (nullable = true)
 |-- number_of_trades: long (nullable = true)
 |-- taker_buy_base_volume: double (nullable = true)
 |-- taker_buy_quote_volume: double (nullable = true)
 |-- ignore: string (nullable = true)



In [88]:
df_bronze.columns

['open_time',
 'open',
 'high',
 'low',
 'close',
 'volume',
 'close_time',
 'quote_asset_volume',
 'number_of_trades',
 'taker_buy_base_volume',
 'taker_buy_quote_volume',
 'ignore']

In [89]:
df_bronze.describe().show()

+-------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------------+----------------------+------+
|summary|              open|              high|               low|             close|           volume|quote_asset_volume|  number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|
+-------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------------+----------------------+------+
|  count|               600|               600|               600|               600|              600|               600|               600|                  600|                   600|   600|
|   mean| 93070.08556666668| 93084.20378333335| 93055.98494999995|  93070.1422166667|5.902528833333327| 549296.6430280326|1694.1466666666668|    2.621530233333333|     243988.0310935778|   0.0|
| stddev|119.90986944226844|11

In [90]:
from pyspark.sql.functions import col, sum as spark_sum, when
df_bronze.select([
    spark_sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
      for c in df_bronze.columns
      ]).show()

+---------+----+----+---+-----+------+----------+------------------+----------------+---------------------+----------------------+------+
|open_time|open|high|low|close|volume|close_time|quote_asset_volume|number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|
+---------+----+----+---+-----+------+----------+------------------+----------------+---------------------+----------------------+------+
|        0|   0|   0|  0|    0|     0|         0|                 0|               0|                    0|                     0|     0|
+---------+----+----+---+-----+------+----------+------------------+----------------+---------------------+----------------------+------+



26/01/21 14:22:21 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:359)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:81)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:674)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1363)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:356)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.scala:18)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:1941

In [91]:
print("Nombres des lingne:", df_bronze.count())

Nombres des lingne: 600


In [92]:
uniques_lignes = df_bronze.dropDuplicates().count()
print(f"Nombre de lignes uniques :{uniques_lignes}")

Nombre de lignes uniques :600


In [93]:
# decaler la colonne close de 10 lignes

window = Window.orderBy("open_time")

df_silver = df_bronze.withColumn("close_t_plus_10", F.lead("close", 10).over(window))


In [94]:
# Calcule du return 
window = Window.orderBy("open_time")

df_silver = df_silver.withColumn(
    "return_1m",
    (F.col("close") - F.lag("close", 1).over(window)) / F.lag("close", 1).over(window)
)

df_silver.select("open_time", "close", "return_1m").show(5)


+-------------------+--------+--------------------+
|          open_time|   close|           return_1m|
+-------------------+--------+--------------------+
|2026-01-19 09:53:00| 93001.8|                NULL|
|2026-01-19 09:54:00|93014.94|1.412875879821618...|
|2026-01-19 09:55:00|93022.36| 7.97721312296525E-5|
|2026-01-19 09:56:00|93019.77|-2.78427681258195...|
|2026-01-19 09:57:00|93016.23|-3.80564260695134...|
+-------------------+--------+--------------------+
only showing top 5 rows


26/01/21 14:22:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [95]:
#  la moyenne des prix de clôture sur les 5 et 10 dernières minutes 

window_5 = Window.orderBy("open_time").rowsBetween(-4, 0)
window_10 = Window.orderBy("open_time").rowsBetween(-9, 0)
  
df_silver = df_silver.withColumn("MA_5", F.avg("close").over(window_5))
df_silver = df_silver.withColumn("MA_10", F.avg("close").over(window_10))

In [96]:
# Volume et intensité de trading

df_silver = df_silver.withColumn("taker_ratio", 
      F.col("taker_buy_base_volume") / F.col("volume"))

In [97]:
df_silver.show()


+-------------------+--------+--------+--------+--------+--------+--------------------+------------------+----------------+---------------------+----------------------+------+---------------+--------------------+-----------------+-----------------+--------------------+
|          open_time|    open|    high|     low|   close|  volume|          close_time|quote_asset_volume|number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|close_t_plus_10|           return_1m|             MA_5|            MA_10|         taker_ratio|
+-------------------+--------+--------+--------+--------+--------+--------------------+------------------+----------------+---------------------+----------------------+------+---------------+--------------------+-----------------+-----------------+--------------------+
|2026-01-19 09:53:00|93046.35|93046.35|92997.65| 93001.8| 9.10532|2026-01-19 09:53:...|    847027.2616431|            2024|              2.08878|        194272.7849138|     0|       92965.51

26/01/21 14:22:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [98]:
df_silver.select([
    spark_sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
      for c in df_silver.columns
      ]).show()

+---------+----+----+---+-----+------+----------+------------------+----------------+---------------------+----------------------+------+---------------+---------+----+-----+-----------+
|open_time|open|high|low|close|volume|close_time|quote_asset_volume|number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|close_t_plus_10|return_1m|MA_5|MA_10|taker_ratio|
+---------+----+----+---+-----+------+----------+------------------+----------------+---------------------+----------------------+------+---------------+---------+----+-----+-----------+
|        0|   0|   0|  0|    0|     0|         0|                 0|               0|                    0|                     0|     0|             10|        1|   0|    0|          0|
+---------+----+----+---+-----+------+----------+------------------+----------------+---------------------+----------------------+------+---------------+---------+----+-----+-----------+



26/01/21 14:22:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [99]:
#drop rows where target is null 
df_silver = df_silver.na.drop(subset=["close_t_plus_10", "return_1m"])

In [100]:
df_silver.select([
    spark_sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
      for c in df_silver.columns
      ]).show()

+---------+----+----+---+-----+------+----------+------------------+----------------+---------------------+----------------------+------+---------------+---------+----+-----+-----------+
|open_time|open|high|low|close|volume|close_time|quote_asset_volume|number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|close_t_plus_10|return_1m|MA_5|MA_10|taker_ratio|
+---------+----+----+---+-----+------+----------+------------------+----------------+---------------------+----------------------+------+---------------+---------+----+-----+-----------+
|        0|   0|   0|  0|    0|     0|         0|                 0|               0|                    0|                     0|     0|              0|        0|   0|    0|          0|
+---------+----+----+---+-----+------+----------+------------------+----------------+---------------------+----------------------+------+---------------+---------+----+-----+-----------+



26/01/21 14:22:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [101]:
uniques_lignes = df_silver.dropDuplicates().count()
print(f"Nombre de lignes uniques :{uniques_lignes}")

26/01/21 14:22:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


Nombre de lignes uniques :589


In [102]:
from pyspark.sql import functions as F

# Calculer les quartiles et l'IQR
quantiles = df_silver.select(
    F.expr('percentile_approx(close_t_plus_10, 0.25)').alias('Q1'),
    F.expr('percentile_approx(close_t_plus_10, 0.75)').alias('Q3')
).collect()[0]

Q1 = quantiles['Q1']
Q3 = quantiles['Q3']
IQR = Q3 - Q1

# Définir les bornes
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Q1: {Q1}, Q3: {Q3}, IQR: {IQR}")
print(f"Bornes: [{lower_bound}, {upper_bound}]")

# Détecter les outliers
df_with_outliers = df_silver.withColumn(
    'is_outlier',
    F.when(
        (F.col('close_t_plus_10') < lower_bound) | (F.col('close_t_plus_10') > upper_bound),
        True
    ).otherwise(False)
)

# Compter les outliers
outlier_count = df_with_outliers.filter(F.col('is_outlier') == True).count()
print(f"Nombre d'outliers: {outlier_count}")

# Afficher les outliers
df_with_outliers.filter(F.col('is_outlier') == True).show()

26/01/21 14:22:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


Q1: 92998.27, Q3: 93146.1, IQR: 147.83000000000175
Bornes: [92776.525, 93367.845]


26/01/21 14:22:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 1

Nombre d'outliers: 8
+-------------------+--------+--------+--------+--------+--------+--------------------+------------------+----------------+---------------------+----------------------+------+---------------+--------------------+-----------------+-----------------+--------------------+----------+
|          open_time|    open|    high|     low|   close|  volume|          close_time|quote_asset_volume|number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|close_t_plus_10|           return_1m|             MA_5|            MA_10|         taker_ratio|is_outlier|
+-------------------+--------+--------+--------+--------+--------+--------------------+------------------+----------------+---------------------+----------------------+------+---------------+--------------------+-----------------+-----------------+--------------------+----------+
|2026-01-19 14:26:00|92813.52|92842.91|92813.51|92838.05| 4.28239|2026-01-19 14:26:...|    397495.9354988|            1525|             

26/01/21 14:22:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [103]:
df_silver = df_silver.drop('ignore')
df_silver.show()

26/01/21 14:22:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------------+--------+--------+--------+--------+--------+--------------------+------------------+----------------+---------------------+----------------------+---------------+--------------------+-----------------+-----------------+--------------------+
|          open_time|    open|    high|     low|   close|  volume|          close_time|quote_asset_volume|number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|close_t_plus_10|           return_1m|             MA_5|            MA_10|         taker_ratio|
+-------------------+--------+--------+--------+--------+--------+--------------------+------------------+----------------+---------------------+----------------------+---------------+--------------------+-----------------+-----------------+--------------------+
|2026-01-19 09:54:00| 93001.8|93022.23|93001.79|93014.94| 5.62556|2026-01-19 09:54:...|    523245.8306426|            1064|              3.42885|        318907.7899883|       92975.79|1.412875879821618...|      

26/01/21 14:22:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:22:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [ ]:
df_silver.coalesce(1)\
    .write.mode("overwrite")\
    .parquet("../data/silver/silver_dataset")


26/01/21 14:23:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:23:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:23:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:23:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 14:23:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


26/01/21 14:23:10 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:359)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:131)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:707)
	at org.apache.spark.storage.BlockManagerMasterE